In [ ]:
import pandas as pd

# CSV 파일 경로 설정
csv_file_path = 'yolov8_aug_changed.csv'

classes = ["General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing"]


df = pd.read_csv(csv_file_path)
rows = []

# 각 이미지에 대한 bbox 정보 파싱
for index, row in df.iterrows():
    # PredictionString이 NaN이 아닐 경우에만 처리
    if pd.notna(row['PredictionString']):
        predictions = row['PredictionString'].strip().split(' ')
        image_id = row['image_id']

        # 각 bbox를 추출하여 행 추가
        for i in range(0, len(predictions), 6):
            if i + 5 < len(predictions):
                class_id = int(predictions[i])
                class_name = classes[class_id]
                confidence = float(predictions[i + 1])
                x1 = float(predictions[i + 2])
                y1 = float(predictions[i + 3])
                x2 = float(predictions[i + 4])
                y2 = float(predictions[i + 5])
                rows.append([image_id, class_id, class_name, confidence, x1, x2, y1, y2])

# 새로운 DataFrame 생성
columns = ['image_id', 'class_id', 'class_name', 'confidence', 'x_min', 'y_min', 'x_max', 'y_max']
bbox_pred = pd.DataFrame(rows, columns=columns)

# 결과 출력
bbox_pred


In [ ]:
# test 파일 개수 확인 = 4871
len(df)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import random
import os

# 데이터셋 경로 설정
BASE_DIR = './dataset'

# 바운딩 박스 색상 설정 (클래스명에 따라)
class_colors = {
    "General trash": "red", 
    "Paper": "orange", 
    "Paper pack": "yellow", 
    "Metal": "green", 
    "Glass": "blue", 
    "Plastic": "purple", 
    "Styrofoam": "cyan", 
    "Plastic bag": "magenta", 
    "Battery": "pink", 
}

# bbox를 포함하는 sample
def plot_sample_with_bbox(image_path, bbox):
    # 이미지 로드
    img = Image.open(image_path)
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # 이미지 표시
    ax.imshow(img)
    ax.axis('off')
    
    # Bbox 그리기
    for _bbox_info in bbox:
        x, y, w, h, class_name = _bbox_info
        color = class_colors.get(class_name, 'yellow')  # 기본 색상은 노란색
        rect = patches.Rectangle((x, y), w, h, linewidth=2, edgecolor=color, facecolor='none')
        ax.add_patch(rect)
        ax.text(x, y, f"{class_name}", fontsize=7, color='white', bbox=dict(facecolor=color, alpha=0.3))

    plt.title(os.path.basename(image_path), loc='center', pad=5)
    plt.axis('off')
    plt.show()
    
# bbox anchor을 coco 형태로 재구성
def bbox_anchor(Dataframe):
    # x, y, w, h
    bbox_anchor_info = [Dataframe['x_min'], 
                        Dataframe['y_min'], 
                        Dataframe['x_max'] - Dataframe['x_min'], 
                        Dataframe['y_max'] - Dataframe['y_min']]
    
    return bbox_anchor_info

# 데이터셋에서 랜덤하게 샘플 선택
sample_index = random.randint(0, len(bbox_pred) - 1)
sample_row = bbox_pred.iloc[sample_index]

# 이미지 경로와 Bbox 정보 추출
image_path = os.path.join(BASE_DIR, sample_row['image_id'])
sample_img_df = bbox_pred[bbox_pred['image_id'] == sample_row['image_id']]

# 해당 sample의 모든 bbox anchor
bbox_set = []

for idx in range(len(sample_img_df)):
    bbox_info = bbox_anchor(sample_img_df.iloc[idx])
    class_name = sample_img_df.iloc[idx]['class_name']
    bbox_set.append((*bbox_info, class_name))

# 샘플 이미지와 Bbox 표시
plot_sample_with_bbox(image_path, bbox_set)

print(f"Image ID: {sample_row['image_id']}")
print(f"Bbox 개수: {len(bbox_set)}")


In [ ]:
# 샘플 이미지 정보 출력
print(f"Sample Row: {sample_row}")

# 샘플 이미지 DataFrame 확인
print("Sample Image DataFrame:")
sample_img_df

In [ ]:
import pandas as pd

# CSV 파일 경로 설정
csv_file_path = 'cascade_rcnn_no_aug.csv'

# CSV 파일 읽기
df = pd.read_csv(csv_file_path)

# NaN 값이 있는 열과 그 개수 출력
nan_summary = df.isna().sum()
print(nan_summary[nan_summary > 0])

# 특정 열에서 NaN 값이 있는 행 출력 (예: 'PredictionString' 열) ==> bbox 없음
nan_rows = df[df['PredictionString'].isna()]
print(nan_rows)


In [ ]:
import os
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from tqdm import tqdm  # tqdm 임포트 추가

# 데이터셋 경로 설정
BASE_DIR = './dataset'
OUTPUT_DIR = './submission_bbox'  # 결과 저장할 경로

# 결과 저장 디렉토리 생성
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 바운딩 박스 색상 설정 (클래스명에 따라)
class_colors = {
    "General trash": "red", 
    "Paper": "orange", 
    "Paper pack": "yellow", 
    "Metal": "green", 
    "Glass": "blue", 
    "Plastic": "purple", 
    "Styrofoam": "cyan", 
    "Plastic bag": "magenta", 
    "Battery": "pink", 
    "Clothing": "brown"
}

# bbox anchor을 coco 형태로 재구성
def bbox_anchor(Dataframe):
    # x, y, w, h
    bbox_anchor_info = [
        Dataframe['x_min'], 
        Dataframe['y_min'], 
        Dataframe['x_max'] - Dataframe['x_min'], 
        Dataframe['y_max'] - Dataframe['y_min']
    ]
    return bbox_anchor_info

# 이미지 ID에서 숫자 부분 추출
def extract_image_id(image_id):
    return int(os.path.splitext(os.path.basename(image_id))[0])

# 모든 이미지에 bbox 적용
def save_images_with_bbox(dataframe):
    for image_id in tqdm(dataframe['image_id'].unique(), desc="Processing images"):
        image_path = os.path.join(BASE_DIR, image_id)
        sample_img_df = dataframe[dataframe['image_id'] == image_id]

        # 이미지 로드
        img = Image.open(image_path)
        fig, ax = plt.subplots(figsize=(10, 6))
        ax.imshow(img)
        ax.axis('off')

        # 해당 sample의 모든 bbox anchor
        bbox_set = []

        for idx in range(len(sample_img_df)):
            bbox_info = bbox_anchor(sample_img_df.iloc[idx])
            class_name = sample_img_df.iloc[idx]['class_name']
            bbox_set.append((*bbox_info, class_name))

            # Bbox 색상 설정
            color = class_colors.get(class_name, 'yellow')  # 기본 색상은 노란색

            # Bbox 그리기
            x, y, w, h = bbox_info
            rect = patches.Rectangle((x, y), w, h, linewidth=2, edgecolor=color, facecolor='none')
            ax.add_patch(rect)
            ax.text(x, y, f"{class_name}", fontsize=7, color='white', bbox=dict(facecolor=color, alpha=0.4))

        # 이미지 제목 추가
        plt.title(os.path.basename(image_path), loc='center', pad=5)
        
        # JPG 파일로 저장
        output_image_path = os.path.join(OUTPUT_DIR, os.path.splitext(image_id)[0][-4:] + '_bbox.jpg')
        plt.savefig(output_image_path, bbox_inches='tight', pad_inches=0)
        plt.close(fig)  # 메모리 절약을 위해 닫기

# 모든 이미지에 바운딩 박스를 그려서 저장
save_images_with_bbox(bbox_pred)

print(f"All images with bounding boxes have been saved to {OUTPUT_DIR}.")
